
<table width="100%"  alignt="left" position="absolute"  style="font-family:verdana;font-size:10px"  >
<colgroup>
<col  style="width: 180px">
</colgroup>
<thead>
  <tr >
    <td   rowspan="5">
    <img  src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQG0LSU20JCX7v7iAIc0FrPRJOloXv20rr1ZRV2DE6IuDXZwVc5Vf9awP7-mfHQXQl7BSI&usqp=CAU" alt="drawing" style="width:180px;"/>
    </td>
    <td style="font-size:10px" ><b>Universidad Internacional de Valencia</b></td>
  </tr>
  <tr>
    <td  style="font-size:9px"><b>Maestria en Big Data y Ciencia de Datos</b></td>
  </tr>
  <tr  style="font-size:9px">
    <td  >Minería de Datos - 2022</td>
  </tr>
  <tr  style="font-size:9px">
    <td  >Enrique Abraham Magnani Quiñonez</td>
  </tr>
  
</thead>
</table>


In [ ]:
pip install pandas_profiling

In [ ]:
import numpy as np # librería para manejar datos vectorizados
import warnings
import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import json # librería para manejar archivos JSON 
from sklearn.preprocessing import MinMaxScaler
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas
import pandas_profiling
#descomentar durante el desarrollo
warnings.filterwarnings("ignore")
print('Librerias importadas.')

### Funciones

Carga via **Github** DataSet de datos transaccionales

In [ ]:
#Carga de data transaccional
def fn_data_df_transacciones():
    try:
        url_df_data_tarjeta='https://github.com/i3nriq/VIU_MineriadeDatos/blob/f0bc3c6aedda3082e55a6cd74336e9002a4cb591/Data/DS_Data_TJT.csv?raw=true'
        import_df_data_tarjeta = pd.read_csv(url_df_data_tarjeta, encoding='latin-1')
        print('Carga via Github..')
    except:
        print('Datos no cargados..')
        import_df_data_tarjeta= None
    return(import_df_data_tarjeta)

Carga via **Github** DataSet de MCG.

In [ ]:
#Carga de data MCG
def fn_data_df_mcg():
    try:
        url_df_data_mcg='https://github.com/i3nriq/VIU_MineriadeDatos/blob/main/Data/DS_MCC_MCG.csv?raw=true'
        import_df_data_mcg = pd.read_csv(url_df_data_mcg, encoding='latin-1')
        print('Carga via Github..')
    except:
        print('Datos no cargados..')
        import_df_data_mcg= None
    return(import_df_data_mcg)

Carga via **Github** DataSet de rangos de edades.

In [ ]:
#Carga de data MCG
def fn_data_df_rango_edad():
    try:
        url_df_data_rango_edad='https://github.com/i3nriq/VIU_MineriadeDatos/blob/main/Data/DS_RAN_EDAD.csv?raw=true'
        import_df_data_rango_edad = pd.read_csv(url_df_data_rango_edad, encoding='latin-1')
        print('Carga via Github..')
    except:
        print('Datos no cargados..')
        import_df_data_rango_edad= None
    return(import_df_data_rango_edad)

Top de catagorias MCG

In [ ]:
def retornat_top_mcg(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

Analisis de DataFrame

In [ ]:
def fn_descripcion_dataFrame(i_df_desc):
    print("Dimensiones dataframe: ",i_df_desc.shape)
    print("")
    print(i_df_desc.dtypes)

## Variables globales

#### Parametros

In [ ]:
json_prioridad_metal='{"BLACK":"1","INFINITE":"2","SIGNATURE":"3","PLATINUM":"4","ORO":"5","CLASICA":"6"}'

# Carga de datos

In [ ]:
#df_categorias_config=pd.merge(df_categorias_config, ( pd.DataFrame (abc_series, columns = ['Caracter'])), left_index=True, right_index=True)
json_prioridad_metal = json.loads(json_prioridad_metal)

df_prioridad_metal = pd.DataFrame.from_dict(json_prioridad_metal, orient="index")
df_prioridad_metal.rename(columns={0:'IND_PRIORIDAD'},inplace=True)

df_prioridad_metal.index.name = 'METAL_TJT'
df_prioridad_metal.reset_index(inplace=True)

Data de transaccion de **VISA**

In [ ]:
df_data_tarjeta=fn_data_df_transacciones()

Data de catalogos

In [ ]:
df_data_mcg=fn_data_df_mcg()

In [ ]:
df_data_df_rango_edad=fn_data_df_rango_edad()

In [ ]:
fn_descripcion_dataFrame(df_data_tarjeta)
df_data_tarjeta.head()

In [ ]:
fn_descripcion_dataFrame(df_data_mcg)

In [ ]:
fn_descripcion_dataFrame(df_data_df_rango_edad)

Extraer datos de la transaccion de VISA

In [ ]:
df_data_tarjeta['DETALLE_MOVIMIENTO_VISA'] = df_data_tarjeta['DETALLE_MOVIMIENTO_VISA'].str.replace('[', '>>')
df_data_tarjeta['DETALLE_MOVIMIENTO_VISA'] = df_data_tarjeta['DETALLE_MOVIMIENTO_VISA'].str.replace(']', '<<')

pattern = '|'.join(['THTRX>>','<<-THTR','<<-FDTRX','MTRX:>>','FDTRX>>', '<<'])

df_data_tarjeta['MONTO_TRX']=df_data_tarjeta['DETALLE_MOVIMIENTO_VISA'].str.extract(r'(?P<DETALLE_MOVIMIENTO_VISA>MTRX:>>.*.<<-FDTRX)')#.replace("RAIL","", inplace=True)
df_data_tarjeta['MONEDA_TRX']=df_data_tarjeta['DETALLE_MOVIMIENTO_VISA'].str.extract(r'(?P<DETALLE_MOVIMIENTO_VISA>FDTRX>>.*.<<-THTR)')#.replace("RAIL","", inplace=True)
df_data_tarjeta['TASA_TRX']=df_data_tarjeta['DETALLE_MOVIMIENTO_VISA'].str.extract(r'(?P<DETALLE_MOVIMIENTO_VISA>THTRX>>.*.<<)')

del df_data_tarjeta['DETALLE_MOVIMIENTO_VISA']

df_data_tarjeta['MONTO_TRX']=df_data_tarjeta['MONTO_TRX'].str.replace(pattern, '')
df_data_tarjeta['MONEDA_TRX']=df_data_tarjeta['MONEDA_TRX'].str.replace(pattern, '')
df_data_tarjeta['TASA_TRX']=df_data_tarjeta['TASA_TRX'].str.replace(pattern, '')

In [ ]:
df_data_tarjeta[df_data_tarjeta['C_RN']==4285].head()

In [ ]:
fn_descripcion_dataFrame(df_data_tarjeta)
df_data_tarjeta.head()

## Valores perdidos

In [ ]:
#df_data_tarjeta['DESCRIPCION_MCG'].isnull().values.any()
#df_data_tarjeta['DESCRIPCION_MCG'].value_counts()

In [ ]:
df_data_tarjeta.isna().sum()

### Eliminar columnas

Datos transaccionales

In [ ]:
df_data_tarjeta = df_data_tarjeta[df_data_tarjeta['MONTO_TRX'].notna()]
df_data_tarjeta = df_data_tarjeta[df_data_tarjeta['MONEDA_TRX'].notna()]
df_data_tarjeta = df_data_tarjeta[df_data_tarjeta['TASA_TRX'].notna()]

Datos de Catalogo MCG

In [ ]:
df_data_mcg = df_data_mcg[df_data_mcg['DESC_MCC'].notna()]
df_data_mcg = df_data_mcg[df_data_mcg['DESCRIPCION_MCG'].notna()]

Identificadores criticos

In [ ]:
#Numero de cuenta
df_data_tarjeta = df_data_tarjeta[df_data_tarjeta['C_RN'].notna()]

#Codigo de cliente
df_data_tarjeta = df_data_tarjeta[df_data_tarjeta['CL_RN'].notna()]

In [ ]:
df_data_tarjeta[df_data_tarjeta['C_RN']==4285].head(1000)

### Convertir Tipos de datos

In [ ]:
#Fechas
df_data_tarjeta = df_data_tarjeta.astype({"FECHA_REAL_MOV":"datetime64[ns]","FECHA_NACIMIENTO":"datetime64[ns]","FEC_COSECHA":"datetime64[ns]"})

#Daatos Transaccionales
df_data_tarjeta = df_data_tarjeta.astype({"MONTO_TRX":"float64","MONEDA_TRX":"int64","TASA_TRX":"float64"})

In [ ]:
fn_descripcion_dataFrame(df_data_tarjeta)
df_data_tarjeta.head()

Conversion de indicador de prioridad de METAL

In [ ]:
#Convertir indicador para modelo
df_prioridad_metal = df_prioridad_metal.astype({"IND_PRIORIDAD":"int64"})

In [ ]:
fn_descripcion_dataFrame(df_prioridad_metal)
df_prioridad_metal.head()

### Nuevas variables

Edad

In [ ]:
now = pd.Timestamp('now')
df_data_tarjeta['Edad']=(now-df_data_tarjeta['FECHA_NACIMIENTO']).astype('<m8[Y]')  


Estado Civil

In [ ]:
#Simplificar Casados y Unidos = Casad@ de lo contrario Solter@
df_data_tarjeta["ESTADO_CIVIL"] = np.where(((df_data_tarjeta["ESTADO_CIVIL"] =='U')|(df_data_tarjeta["ESTADO_CIVIL"] =='C')), 'C', 'S')

In [ ]:
df_data_tarjeta['ESTADO_CIVIL'].unique()

Se adiciona el agrupador de comercio

In [ ]:
df_data_tarjeta=(pd.merge(df_data_tarjeta, df_data_mcg[['TIPO_RUBRO','DESCRIPCION_MCG']], left_on='TIPO_RUBRO', right_on='TIPO_RUBRO'))

In [ ]:
df_data_tarjeta[['C_RN','CL_RN','TIPO_RUBRO','DESCRIPCION_MCG']].head(3)

In [ ]:
df_data_tarjeta=(pd.merge(df_data_tarjeta, df_prioridad_metal, left_on='METAL_TJT', right_on='METAL_TJT'))

In [ ]:
df_data_tarjeta[['C_RN','CL_RN','METAL_TJT','IND_PRIORIDAD']].head(3)

Valor dolarizado de la transaccion, donde VISA envia en moneda loca y extranjera (distinta al dolar) convierte automaticamente a dolares.

In [ ]:
#Dolarizamos en base al atributo de tasa de cambio, donde 320 es quetzales y 840 es dolares
df_data_tarjeta['MONTO_DOLAR']=np.where(df_data_tarjeta['MONEDA_TRX']== 320, round((df_data_tarjeta['MONTO_TRX']/df_data_tarjeta['TASA_TRX']),2), round(df_data_tarjeta['MONTO_TRX'],2))

In [ ]:
df_data_tarjeta[['C_RN','CL_RN','MONTO_TRX','MONEDA_TRX','TASA_TRX','MONTO_DOLAR']].head(3)

Año cosecha

In [ ]:
df_data_tarjeta['ANIO_COSECHA']=df_data_tarjeta['FEC_COSECHA'].dt.year 

Se valida que no existan **valores perdidos**

In [ ]:
df_data_tarjeta.isna().sum()

In [ ]:
fn_descripcion_dataFrame(df_data_tarjeta)
df_data_tarjeta.head()

## Reconstruccion

#### Fecha

In [ ]:
#Obtener la edad promedio por metal y asignar, las politicas del banco regulan el otorgar el plastico en base a la edad o record crediticio.
df_edad_metal=df_data_tarjeta.drop_duplicates(['Edad','METAL_TJT'])[['Edad','METAL_TJT']]
#Eliiminamos las observaciones que necesitamos reconstruir
df_edad_metal = df_edad_metal[df_edad_metal['Edad'].notna()]
#Obtener la edad Promedio por metal
df_edad_metal=df_edad_metal.groupby("METAL_TJT", as_index=False).agg({"Edad":"mean"})
df_edad_metal['Edad']=round(df_edad_metal['Edad'],0)
df_edad_metal.rename(columns={'Edad':'Edad_promedio'},inplace=True)
df_edad_metal

Reemplazar los valores nulos por Edad Promedio

In [ ]:
d_ex=df_data_tarjeta#[df_data_tarjeta['Edad'].isna()].head()
df_data_tarjeta=(pd.merge(df_data_tarjeta, df_edad_metal, left_on='METAL_TJT', right_on='METAL_TJT'))
df_data_tarjeta ['Edad'] =np.where(df_data_tarjeta['Edad'].isna(), df_data_tarjeta['Edad_promedio'], df_data_tarjeta['Edad'])
del df_data_tarjeta['Edad_promedio']
del df_data_tarjeta['FECHA_NACIMIENTO']
df_data_tarjeta.isna().sum()

In [ ]:
df_data_tarjeta.head()

#### Exportar datos modelo - Regresion Multiple

# Modelado de Datos

In [ ]:
 df_data_tarjeta.to_csv('Data\DF_Data_transaccional.csv', index=False)

Obtener los montos por transaccion de los clientes

In [ ]:
#df_agrupado_metal=df_data_tarjeta.groupby("METAL_TJT", as_index=False).agg({"Edad":"mean"})
resumen_categorias_top=df_data_tarjeta.groupby(['C_RN','DESCRIPCION_MCG'])[["MONTO_DOLAR"]].sum()

resumen_categorias_top.reset_index(drop=False,inplace=True)
resumen_categorias_top.head()


Analisis de cuantos **comercios distintos** en promedio presenta la poblacion

In [ ]:
df_agrupado_por_comercio=resumen_categorias_top.groupby("C_RN", as_index=False).agg({"DESCRIPCION_MCG":"count"})
df_agrupado_por_comercio['DESCRIPCION_MCG'].describe()

Se determina implementar unicamente el comercio con mayor consumo del top 3 que se calculara debibo a la volumetria de transacciones.

In [ ]:
resumen_categorias_cliente=df_data_tarjeta.groupby(['CL_RN','DESCRIPCION_MCG'])[["MONTO_DOLAR"]].sum()
resumen_categorias_cliente.reset_index(drop=False,inplace=True)
resumen_categorias_cliente.head()

Pivot de informacion

In [ ]:
df_consumos_pivot = resumen_categorias_cliente.pivot(index="CL_RN", columns="DESCRIPCION_MCG", values="MONTO_DOLAR")\
       .reset_index().rename_axis(None, axis=1)
df_consumos_pivot=df_consumos_pivot.fillna('0')
df_consumos_pivot.set_index("CL_RN", inplace = True)

In [ ]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_consumos_pivot.values)
df_consumos_pivot = pd.DataFrame(scaled_features, index=df_consumos_pivot.index, columns=df_consumos_pivot.columns)

In [ ]:
df_consumos_pivot.reset_index(drop=False,inplace=True)
df_consumos_pivot.head()

### Ponderar de forma descendente

In [ ]:
num_top = 3
indicators = ['ER', 'DO', 'ER']

# crear las columnas acorde al numero de grupos de comercios con mayor consumo
columnas = ['CL_RN']
for ind in np.arange(num_top):
    try:
        columnas.append('{}{}_MCG'.format(ind+1, indicators[ind]))
    except:
        columnas.append('{}th MCG'.format(ind+1))
        

df_clientes_mcg_ordenado = pd.DataFrame(columns=columnas)
df_clientes_mcg_ordenado['CL_RN'] = df_consumos_pivot['CL_RN']

for ind in np.arange(df_consumos_pivot.shape[0]):
    w_rank=    retornat_top_mcg(df_consumos_pivot.iloc[ind, 1:], num_top)   
    df_clientes_mcg_ordenado.iloc[ind, 1:] =w_rank
    

df_clientes_mcg_ordenado.head()

In [ ]:
#df_clientes_mcg_ordenado.head()
df_clientes_mcg_ordenado=df_clientes_mcg_ordenado[['CL_RN','1ER_MCG']]
df_clientes_mcg_ordenado.rename(columns={'1ER_MCG':'COMERCIO_CON_MAS_CONSUMO'},inplace=True)
df_clientes_mcg_ordenado.head()

### Variables por cliente

Se obtiene las variables por clientes, con los siguiente cristerior: 
*1.* ponderando el pago de contado coomo N contagiando los registros del cliente durante el perioro evaluado.
*2.* Obtenemos la maxima categoria obtenida por cliente entre tu cartera de cartejas.

In [ ]:
df_rsm_cliente=df_data_tarjeta.groupby(['CL_RN','LIM_CREDITO','ANIO_COSECHA','COD_MORA','Edad']).agg(PAGO_CONTADO=('PAGO_CONTADO',min),METAL_TJT=('IND_PRIORIDAD',max))
df_rsm_cliente.reset_index(drop=False,inplace=True)
df_rsm_cliente.head()

### Rangos de edades

In [ ]:
df_data_df_rango_edad.head()

In [ ]:
a_pivot = df_rsm_cliente['Edad'].values
bl_max = df_data_df_rango_edad['RANGO_FIN'].values
bl_min = df_data_df_rango_edad['RANGO_INI'].values
i, j = np.where((a_pivot[:, None] >= bl_min) & (a_pivot[:, None] <=bl_max))

Integrar rango de fechas **between**

In [ ]:
#LEFT JOIN 
#Tip: .append en adelante
df_left_join=pd.concat([
    df_rsm_cliente.loc[i, :].reset_index(drop=True),
    df_data_df_rango_edad.loc[j, :].reset_index(drop=True)
], axis=1).append(
    df_rsm_cliente[~np.in1d(np.arange(len(df_rsm_cliente)), np.unique(i))],
    ignore_index=True, sort=False
)

In [ ]:
fn_descripcion_dataFrame(df_left_join)

In [ ]:
df_cliente=(pd.merge(df_clientes_mcg_ordenado, df_left_join[['CL_RN','LIM_CREDITO','ANIO_COSECHA','COD_MORA','PAGO_CONTADO','METAL_TJT','COD_RANGO_EDAD','Edad']], left_on='CL_RN', right_on='CL_RN'))
fn_descripcion_dataFrame(df_cliente)
print('\nValidamos campos perdidos.')
df_cliente.isna().sum()

In [ ]:
df_cliente.head()

### Guardar dataframe to CSV

In [ ]:
 df_cliente.to_csv('Data\DF_Data_depurada.csv', index=False)

In [ ]:
profile = ProfileReport(df_cliente, title="Analisis - Enrique Magnani - Pandas Profile")
profile.to_file("/Reporte_data_depurada.html")